# BTK DATATHON-2025

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df_train = pd.read_csv(r"C:\Users\yusuf\OneDrive\Masaüstü\btk25\data\train.csv")
df_test = pd.read_csv(r"C:\Users\yusuf\OneDrive\Masaüstü\btk25\data\test.csv")
df_submission = pd.read_csv(r"C:\Users\yusuf\OneDrive\Masaüstü\btk25\data\sample_submission.csv")

In [ ]:
df_train

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_train["user_id"].unique()

## Feature Enginering

In [ ]:
def preprocess_df(df):
    """
    Verilen dataframe üzerinde aşağıdaki işlemleri yapar:
    1. event_time → day_type, time_of_day (manuel mapping ile sayısal)
    2. event_type → manuel mapping
    3. product_id, category_id, user_id, user_session → Label Encoding
    4. Orijinal event_time sütunu silinir
    """
    df = df.copy()
    
    # 1️⃣ Event time feature engineering
    df['event_time'] = pd.to_datetime(df['event_time'])
    
    # Weekday / Weekend
    df['day_type'] = df['event_time'].dt.weekday.apply(lambda x: 'Weekend' if x >= 5 else 'Weekday')
    # Manuel mapping: Weekday=0, Weekend=1
    day_type_mapping = {'Weekday': 0, 'Weekend': 1}
    df['day_type'] = df['day_type'].map(day_type_mapping)
    
    # Saat dilimi
    def get_time_of_day(hour):
        if 5 <= hour < 12:
            return 'Morning'
        elif 12 <= hour < 17:
            return 'Afternoon'
        elif 17 <= hour < 21:
            return 'Evening'
        else:
            return 'Night'
    
    df['time_of_day'] = df['event_time'].dt.hour.apply(get_time_of_day)
    # Manuel mapping: Morning=0, Afternoon=1, Evening=2, Night=3
    time_mapping = {'Morning': 0, 'Afternoon': 1, 'Evening': 2, 'Night': 3}
    df['time_of_day'] = df['time_of_day'].map(time_mapping)
    
    # Orijinal event_time sütunu sil
    df = df.drop(columns=['event_time'])
    
    # 2️⃣ event_type manuel mapping
    event_type_mapping = {
        'ADD_CART': 0,
        'VIEW': 1,
        'REMOVE_CART': 2,
        'BUY': 3
    }
    df['event_type'] = df['event_type'].map(event_type_mapping)
    
    # 3️⃣ Label Encoding diğer kategorik sütunlar
    le_cols = ['product_id', 'category_id', 'user_id']
    for col in le_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
    
    return df

In [ ]:
df_train_processed = preprocess_df(df_train)
df_test_processed = preprocess_df(df_test)

In [ ]:
df_train_processed["day_type"].unique()

In [ ]:
df_train_processed

### Model Hazırlık

In [ ]:
X = df_train_processed.drop(columns=['user_session','session_value'])
y = df_train_processed['session_value']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# RandomForest hazır parametrelerle
rf_model = RandomForestRegressor(
    n_estimators=500,          # Ağaç sayısı
    max_depth=None,            # Ağaç derinliği sınırsız
    min_samples_split=5,       # Her düğümde en az örnek sayısı
    min_samples_leaf=2,        # Yaprak başına minimum örnek
    max_features='sqrt',       # Her split için rastgele feature sayısı
    n_jobs=-1,                 # Tüm CPU çekirdeklerini kullan
    random_state=42
)

In [ ]:
# Modeli eğit
rf_model.fit(X_train, y_train)

In [ ]:
# Validation tahmini
y_pred = rf_model.predict(X_val)

In [ ]:
# Metrikler
mse = mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_val_pred)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R^2: {r2:.4f}")